In [1]:
import RingDataset
from Models import CNNModel, RNNGenerator, Distiller, MLP, RNNModel
import os
import numpy as np
import pickle
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import re
import time

In [2]:
testset =  RingDataset.RingDataset('core4ToSlice3_test.pkl', threshold=42)

testloader = DataLoader(testset, batch_size=256, num_workers=4)
classifier_test = CNNModel(42, dim=256).cuda()

gen=RNNGenerator(42, scale=0.25, dim=16, drop=0.0)
assert os.path.isfile('./models/best_{}_{}.pth'.format('cmp', 16))
gen.load_state_dict(torch.load('./models/best_{}_{}.pth'.format('cmp', 16)))



<All keys matched successfully>

In [3]:
def shifter(arr, window=32):
    dup = arr[:,None,:].expand(arr.size(0), arr.size(1)+1, arr.size(1))
    dup2 = dup.reshape(arr.size(0), arr.size(1), arr.size(1)+1)
    shifted = dup2[:,:window,:-window]
    return shifted

In [10]:
class RNNGen2(nn.Module):
    def __init__(self, gen):
        super().__init__()
        self.encoder = gen.encoder
        self.quant = torch.quantization.QuantStub()
        self.resblock = gen.resblock

        self.decoder = gen.decoder
        self.dequant = torch.quantization.DeQuantStub()

    def init_hidden(self, bsz):
        weight = next(self.parameters())
        return weight.new_zeros(2, bsz, 16)
                
    def forward(self, x, hidden):
        x = self.quant(x)
        encoded = self.encoder(x.permute(0,2,1)) #N,C,S -> N,S,C
        hidden = self.quant(hidden)
        res, hidden = self.resblock(encoded, hidden)
        out = encoded + res #N,S,C
        out = self.decoder(out).view(out.size(0),-1)
        #out = out + self.scale*torch.randn_like(out)
        #out = out + noise
        
        return self.dequant(torch.relu(out))
gen2 = RNNGen2(gen)

In [19]:
model = gen2
model.eval()

model_int8 = torch.quantization.quantize_dynamic(
    model, {nn.GRU, nn.Linear}, dtype=torch.qint8
)
print(quantized_model)

input_fp32, _ = next(iter(testloader))
hidden_fp32 = gen2.init_hidden(input_fp32.size(0))
shifted = shifter(input_fp32)

'''
model.qconfig = torch.quantization.get_default_qconfig('fbgemm')

# Prepare the model for static quantization. This inserts observers in
# the model that will observe activation tensors during calibration.
model_fp32_prepared = torch.quantization.prepare(model)

# calibrate the prepared model to determine quantization parameters for activations
# in a real world setting, the calibration would be done with a representative dataset

model_fp32_prepared(shifted, hidden_fp32)

# Convert the observed model to a quantized model. This does several things:
# quantizes the weights, computes and stores the scale and bias value to be
# used with each activation tensor, and replaces key operators with quantized
# implementations.
model_int8 = torch.quantization.convert(model_fp32_prepared)

# run the model, relevant calculations will happen in int8
'''


# run the model, relevant calculations will happen in int8
res = model_int8(shifted, hidden_fp32)
res2 = model(shifted, hidden_fp32)
print(res*256)

RNNGen2(
  (encoder): Sequential(
    (0): DynamicQuantizedLinear(in_features=32, out_features=16, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
    (1): Dropout(p=0.0, inplace=False)
  )
  (quant): QuantStub()
  (resblock): GRU(16, 16, num_layers=2, batch_first=True)
  (decoder): Sequential(
    (0): DynamicQuantizedLinear(in_features=16, out_features=1, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
  )
  (dequant): DeQuantStub()
)
tensor([[  84.5053,   50.7485,    0.0000,  ...,  857.9224,  858.8124,
         1192.4384],
        [   0.0000,    0.0000,    0.0000,  ...,  526.6487,  929.5681,
          857.4774],
        [  99.5083,    0.0000,    0.0000,  ..., 1119.3942, 1075.4023,
         1067.2015],
        ...,
        [  13.8767,    0.0000,    0.0000,  ...,  700.5181,  895.3664,
         1115.3892],
        [   0.0000,    0.0000,    0.0000,  ...,  793.5240,  790.4725,
         1173.6211],
        [ 570.7040,    0.0000,    0.0000,  ...,    0.0000,  539.2359,
         1044

In [18]:
quant = torch.quantization.QuantStub()
a = torch.randn(8)
q = quant(a)
q2 = a.qint8()

AttributeError: 'Tensor' object has no attribute 'qint8'